# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

T. Pfeil  ->  T. Pfeil  |  ['T. Pfeil']
H. Klahr  ->  H. Klahr  |  ['H. Klahr']
S. Li  ->  S. Li  |  ['S. Li']
J. Lian  ->  J. Lian  |  ['J. Lian']


J. Li  ->  J. Li  |  ['J. Li']
S. Li  ->  S. Li  |  ['S. Li']
J. Liu  ->  J. Liu  |  ['J. Liu']
Arxiv has 85 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2406.10335


extracting tarball to tmp_2406.10335...

 done.


T. Pfeil  ->  T. Pfeil  |  ['T. Pfeil']
H. Klahr  ->  H. Klahr  |  ['H. Klahr']


...

extracting tarball to tmp_2406.11602...

 done.
Retrieving document from  https://arxiv.org/e-print/2406.11800


extracting tarball to tmp_2406.11800...

 done.
  0: tmp_2406.11800/report2referee.tex, 119 lines
  1: tmp_2406.11800/SMA_SgrB2_mag.tex, 677 lines


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found 2 candidates with documentclass definition.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Assuming tmp_2406.11800/SMA_SgrB2_mag.tex as main document.
  exec(code_obj, self.user_global_ns, self.user_ns)


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2406.10335-b31b1b.svg)](https://arxiv.org/abs/2406.10335) | **Vertical shear instability with dust evolution and consistent cooling times. On the importance of the initial dust distribution**  |
|| <mark>T. Pfeil</mark>, T. Birnstiel, <mark>H. Klahr</mark> |
|*Appeared on*| *2024-06-18*|
|*Comments*| *6 pages, 3 figures, accepted for publication as a Letter to the Editor in Astronomy and Astrophysics*|
|**Abstract**|            Context. Gas in protoplanetary disks mostly cools via thermal accommodation with dust particles. Thermal relaxation is thus highly sensitive to the local dust size distributions and the spatial distribution of the grains. So far, the interplay between thermal relaxation and gas turbulence has not been dynamically modeled in hydrodynamic simulations of protoplanetary disks with dust. Aims. We aim to study the effects of the vertical shear instability (VSI) on the thermal relaxation times, and vice versa. We are particularly interested in the influence of the initial dust grain size on the VSI and whether the emerging turbulence is sustained over long timescales. Results. We find that the emergence of the VSI is strongly dependent on the initial dust grain size. Coagulation also counteracts the emergence of hydrodynamic turbulence in our simulations, as shown by others before. Starting a simulation with larger grains (100 $\mu$m) generally leads to a less turbulent outcome. While the inner disk regions (within $\sim$ 70 au) develop turbulence in all three simulations, we find that the simulations with larger particles do not develop VSI in the outer disk. Conclusions. Our simulations with dynamically calculated thermal accommodation times based on the drifting and settling dust distribution show that the VSI, once developed in a disk, can be sustained over long timescales, even if grain growth is occurring. The VSI corrugates the dust layer and even diffuses the smaller grains into the upper atmosphere, where they can cool the gas. Whether the instability can emerge for a specific stratification depends on the initial dust grain sizes and the initial dust scale height. If the grains are initially $\gtrsim$ 100 $\mu$m and if the level of turbulence is initially assumed to be low, we find no VSI turbulence in the outer disk regions.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2406.11394-b31b1b.svg)](https://arxiv.org/abs/2406.11394) | **Disk Assembly of the Milky Way Suggested from the Time-resolved Chemical Abundance**  |
|| E. Wang, <mark>J. Lian</mark>, Y. Peng, X. Wang |
|*Appeared on*| *2024-06-18*|
|*Comments*| *11 pages, 4 figures, accepted to ApJ*|
|**Abstract**|            Both simulations and observations suggest that the disk assembly of galaxies is governed by the interplay between coplanar gas inflow, ex-planar gas outflow and in-situ star formation on the disk, known as the leaky accretion disk. This scenario predicts a strong connection between radial distributions of star formation and chemical abundances. The Milky Way, being the sole galaxy where we can reliably measure star formation histories and the corresponding temporally-resolved chemical abundances with individual stars, provides a unique opportunity to scrutinize this scenario. Based on the recent large spectroscopic and photometric surveys of Milky Way stars, we obtain the radial profiles of magnesium abundance ([Mg/H]) and star formation rate (SFR) surface density at different lookback time. We find the radial profiles of [Mg/H] can be well-reproduced using the leaky accretion disk model with only two free parameters for stars formed within 4 Gyr, as well as the flattening at large radii of metallicity profiles traced by HII regions and Cepheids. Furthermore, the constraint effective yield of the Milky Way and nearby galaxies show broad consistency with the theoretical predictions from stellar chemical evolution model with a mass-loading factor of 0-2. These results support that the recent assembly of the Milky Way adheres to the leaky accretion disk scenario, bridging the disk formation of our home galaxy to the big picture of disk formation in the Universe.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2406.11297-b31b1b.svg)](https://arxiv.org/abs/2406.11297) | **Parameter effects on the total intensity of H I Ly{\alpha} line for a modelled coronal mass ejection and its driven shock**  |
|| B. Ying, et al. -- incl., <mark>S. Li</mark> |
|*Appeared on*| *2024-06-18*|
|*Comments*| *22 pages, 9 figures, accepted by Solar Physics*|
|**Abstract**|            The combination of the H I Ly{\alpha} (121.6 nm) line formation mechanism with ultraviolet (UV) Ly{\alpha} and white-light (WL) observations provides an effective method for determining the electron temperature of coronal mass ejections (CMEs). A key to ensuring the accuracy of this diagnostic technique is the precise calculation of theoretical Ly{\alpha} intensities. This study performs a modelled CME and its driven shock via the 3D MHD simulation. We generate synthetic UV and WL images of the CME and shock to quantify the impact of different assumptions on theoretical Ly{\alpha} intensities, such as the incident intensity of the Ly{\alpha} line (Idisk), the geometric scattering function (p({\theta})), and the kinetic temperature (Tn) assumed to be equal to the proton (Tp) or electron (Te) temperatures. By comparing differences of the Ly{\alpha} intensities under these assumptions, we find that: (1) Using the uniform or Carrington maps of the disk Ly{\alpha} emission underestimates the corona Ly{\alpha} intensity (< 10%) compared to the synchronic map, except for a slight overestimate (< 4%) in the partial CME core. The Carrington map yields lower uncertainties than the uniform disk. (2) The geometric scattering process has a minor impact on the Ly{\alpha} intensity, with a maximum relative uncertainty of < 5%. The Ly{\alpha} intensity is underestimated for the most part but overestimated in the CME core. (3) Compared to the assumption Tn = Tp, using Tn = Te leads to more complex relative uncertainties in CME Ly{\alpha} intensity. The CME core and void are both overestimated, with the maximum uncertainty in the core exceeding 50% and the void remaining below 35%. In the CME front, both over- and under-estimates exist with relative uncertainties of < 35%. The electron temperature assumption has a smaller impact on the shock, with an underestimated relative uncertainty of less than 20%.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2406.11602-b31b1b.svg)](https://arxiv.org/abs/2406.11602) | **Association between a Failed Prominence Eruption and the Drainage of Mass from Another Prominence**  |
|| J. Xue, et al. -- incl., <mark>J. Li</mark>, <mark>S. Li</mark> |
|*Appeared on*| *2024-06-18*|
|*Comments*| *15 pages, 7 figures, has been accepted by Solar Physics*|
|**Abstract**|            Sympathetic eruptions of solar prominences have been studied for decades, however, it is usually difficult to identify their causal links. Here we present two failed prominence eruptions on 26 October 2022 and explore their connections. Using stereoscopic observations, the south prominence (PRO-S) erupts with untwisting motions, flare ribbons occur underneath, and new connections are formed during the eruption. The north prominence (PRO-N) rises up along with PRO-S, and its upper part disappears due to catastrophic mass draining along an elongated structure after PRO-S failed eruption. We suggest that the eruption of PRO-S initiates due to a kink instability, further rises up, and fails to erupt due to reconnection with surrounding fields. The elongated structure connecting PRO-N overlies PRO-S, which causes the rising up of PRO-N along with PRO-S and mass drainage after PRO-S eruption. This study suggests that a prominence may end its life through mass drainage forced by an eruption underneath.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2406.11800-b31b1b.svg)](https://arxiv.org/abs/2406.11800) | **Magnetic field in mini starburst complex Sgr B2**  |
|| X. Pan, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2024-06-18*|
|*Comments*| *17 pages, 4 figures, accepted for publication in ApJ*|
|**Abstract**|            We report the first arcsecond-resolution observations of the magnetic field in the mini starburst complex Sgr B2. SMA polarization observations revealed magnetic field morphology in three dense cores of Sgr B2 N(orth), M(ain), and S(outh). The total plane-of-sky magnetic field strengths in these cores are estimated to be 4.3-10.0 mG, 6.2-14.7 mG, and 1.9-4.5 mG derived from the angular dispersion function method after applying the correction factors of 0.21 and 0.5. Combining with analyses of the parsec-scale polarization data from SOFIA, we found that a magnetically supercritical condition is present from the cloud-scale ($\sim$10 pc) to core-scale ($\sim$0.2 pc) in Sgr B2, which is consistent with the burst of star formation activities in the region likely resulted from a multi-scale gravitational collapse from the cloud to dense cores.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2406.10547-b31b1b.svg)](https://arxiv.org/abs/2406.10547) | **Four microlensing giant planets detected through signals produced by minor-image perturbations**  |
|| C. Han, et al. |
|*Appeared on*| *2024-06-18*|
|*Comments*| *10 pages, 12 figures, 7 tables*|
|**Abstract**|            We investigated the nature of the anomalies appearing in four microlensing events KMT-2020-BLG-0757, KMT-2022-BLG-0732, KMT-2022-BLG-1787, and KMT-2022-BLG-1852. The light curves of these events commonly exhibit initial bumps followed by subsequent troughs that extend across a substantial portion of the light curves. We performed thorough modeling of the anomalies to elucidate their characteristics. Despite their prolonged durations, which differ from the usual brief anomalies observed in typical planetary events, our analysis revealed that each anomaly in these events originated from a planetary companion located within the Einstein ring of the primary star. It was found that the initial bump arouse when the source star crossed one of the planetary caustics, while the subsequent trough feature occurred as the source traversed the region of minor image perturbations lying between the pair of planetary caustics. The estimated masses of the host and planet, their mass ratios, and the distance to the discovered planetary systems are $(M_{\rm host}/M_\odot, M_{\rm planet}/M_{\rm J}, q/10^{-3}, \dl/{\rm kpc}) = (0.58^{+0.33}_{-0.30}, 10.71^{+6.17}_{-5.61}, 17.61\pm 2.25,6.67^{+0.93}_{-1.30})$ for KMT-2020-BLG-0757, $(0.53^{+0.31}_{-0.31}, 1.12^{+0.65}_{-0.65}, 2.01 \pm 0.07, 6.66^{+1.19}_{-1.84})$ for KMT-2022-BLG-0732, $(0.42^{+0.32}_{-0.23}, 6.64^{+4.98}_{-3.64}, 15.07\pm 0.86, 7.55^{+0.89}_{-1.30})$ for KMT-2022-BLG-1787, and $(0.32^{+0.34}_{-0.19}, 4.98^{+5.42}_{-2.94}, 8.74\pm 0.49, 6.27^{+0.90}_{-1.15})$ for KMT-2022-BLG-1852. These parameters indicate that all the planets are giants with masses exceeding the mass of Jupiter in our solar system and the hosts are low-mass stars with masses substantially less massive than the Sun.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Unable to locate Ghostscript on paths</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2406.10335/./FiguresPaper2/InnerOuter.png', 'tmp_2406.10335/./FiguresPaper2/zCrit.png', 'tmp_2406.10335/./FiguresPaper2/DtG_tNLTE.png']
copying  tmp_2406.10335/./FiguresPaper2/InnerOuter.png to _build/html/
copying  tmp_2406.10335/./FiguresPaper2/zCrit.png to _build/html/
copying  tmp_2406.10335/./FiguresPaper2/DtG_tNLTE.png to _build/html/
exported in  _build/html/2406.10335.md
    + _build/html/tmp_2406.10335/./FiguresPaper2/InnerOuter.png
    + _build/html/tmp_2406.10335/./FiguresPaper2/zCrit.png
    + _build/html/tmp_2406.10335/./FiguresPaper2/DtG_tNLTE.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\rhog}{\rho_{\mathrm{g}}}$
$\newcommand{\rhod}{\rho_{\mathrm{d}}}$
$\newcommand{\vg}{\vec{v}_{\mathrm{g}}}$
$\newcommand{\vd}{\vec{v}_{\mathrm{d}}}$
$\newcommand{\amax}{a_{\mathrm{max}}}$
$\newcommand{\amin}{a_{\mathrm{min}}}$
$\newcommand{\aint}{a_{\mathrm{int}}}$
$\newcommand{\adr}{a_{\mathrm{dr}}}$
$\newcommand{\vdr}{v_{\mathrm{dr}}}$
$\newcommand{\tfr}{t_\mathrm{fric}}$
$\newcommand{\St}{\mathrm{St}}$
$\newcommand{\Rey}{\mathrm{Re}}$
$\newcommand{\OmK}{\Omega_\text{K}}$
$\newcommand{\der}[2]{\frac{\partial{#1}}{\partial{#2}}}$
$\newcommand{\hd}{h_{\mathrm{d}}}$
$\newcommand{\Mp}{m_{\mathrm{p}}}$
$\newcommand{\Sd}{\Sigma_{\mathrm{d}}}$
$\newcommand{\dpy}{\texttt{DustPy}}$
$\newcommand{\pluto}{\texttt{PLUTO}}$
$\newcommand{\tpop}{\texttt{2pop}}$
$\newcommand{\rev}[1]{{#1}}$
$\newcommand{\revII}[1]{{#1}}$
$\newcommand{\reportI}[1]{{#1}}$
$\newcommand{\corrected}[1]{{#1}}$</div>



<div id="title">

# $\revII{Vertical shear instability with dust evolution and \ consistent cooling times}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2406.10335-b31b1b.svg)](https://arxiv.org/abs/2406.10335)<mark>Appeared on: 2024-06-18</mark> -  _6 pages, 3 figures, accepted for publication as a Letter to the Editor in Astronomy and Astrophysics_

</div>
<div id="authors">

<mark>T. Pfeil</mark>, T. Birnstiel, <mark>H. Klahr</mark>

</div>
<div id="abstract">

**Abstract:** Gas in protoplanetary disks mostly cools via thermal accommodation with dust particles. Thermal relaxation is thus highly sensitive to the local dust size distributions and the spatial distribution of the grains. $\revII{So far, the interplay between thermal relaxation and gas turbulence has not been dynamically modeled in hydrodynamic simulations of protoplanetary disks with dust.}$ $\revII{We aim to study the effects of the vertical shear instability (VSI) on the thermal relaxation times, and vice versa. We are particularly interested in the influence of the initial dust grain size on the VSI and whether the emerging turbulence is sustained over long timescales.}$ We ran three axisymmetric hydrodynamic simulations of a protoplanetary disk including four dust fluids that initially resemble MRN size distributions of different initial grain sizes. From the local dust densities, we calculated the thermal accommodation timescale of dust and gas and used the result as the thermal relaxation time of the gas in our simulation. We included the effect of dust growth by applying the monodisperse dust growth rate and the typical growth limits. We find that the emergence of the VSI is strongly dependent on the initial dust grain size. Coagulation also counteracts the emergence of hydrodynamic turbulence in our simulations, as shown by others before. Starting a simulation with larger grains ( $\SI{100}{\micro \meter}$ ) generally leads to a less turbulent outcome. While the inner disk regions (within $\sim \SI{70}{au}$ ) develop turbulence in all three simulations, we find that the simulations with larger particles do not develop VSI in the outer disk. $\revII{Our simulations with dynamically calculated thermal accommodation times based on the drifting and settling dust distribution}$ show that the VSI, once developed in a disk, can be sustained over long timescales, even if grain growth is occurring. The VSI corrugates the dust layer and even diffuses the smaller grains into the upper atmosphere, where they can cool the gas. Whether the instability can emerge $\revII{for a specific stratification}$ depends on the initial dust grain sizes and the initial dust scale height. If the grains are initially $\gtrsim \SI{100}{\micro \meter}$ and if the level of turbulence is initially assumed to be low, we find $\reportI{no VSI turbulence}$ in the outer disk regions.

</div>

<div id="div_fig1">

<img src="tmp_2406.10335/./FiguresPaper2/InnerOuter.png" alt="Fig1" width="100%"/>

**Figure 1. -** Averaged dust-to-gas ratios and thermal relaxation times in the inner and outer disk.Dust-to-gas ratios and thermal relaxation times radially averaged over the \SIrange{30}{55}{au} inner region and the \SIrange{115}{150}{au} outer region of our simulations. As the VSI develops, it keeps the dust particle at high altitudes and thus retains the necessary cooling times in the inner disk. The outer regions, however, are mostly sedimenting, as the VSI is not active there, except for the case with the smallest initial particle size. (*fig:InnerDisk*)

</div>
<div id="div_fig2">

<img src="tmp_2406.10335/./FiguresPaper2/zCrit.png" alt="Fig2" width="100%"/>

**Figure 2. -** Evolution of the critical VSI height.Evolution of the vertical extend of the VSI susceptible region as a function of time and radius. In the inner disk, VSI quickly develops and stabilizes the settling dust layer in less than \SI{50000}{yr}. In the outer disk, VSI has not yet developed in our simulations. The thin lines show comparison simulations in which the VSI is artificially suppressed. (*fig:zCrit*)

</div>
<div id="div_fig3">

<img src="tmp_2406.10335/./FiguresPaper2/DtG_tNLTE.png" alt="Fig3" width="100%"/>

**Figure 3. -** Dust-to-gas ratios and thermal relaxation times.Snapshots after $\sim$\SI{350000}{yr} of our protoplanetary disk simulations with thermal accommodation times $\rev$II{calculated from the dust distribution for three different initial particle sizes}. Simulations initialized with larger particles and thus more settled dust layers are mostly not VSI active in the outer disk regions. If the VSI, however, can develop in the first place, it can stabilize the dust layer and sustain itself, even if grain growth is considered. (*fig:DtG_tNLTE*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2406.10335"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

97  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
